In [ ]:
import csv, json, math, os
from osgeo import ogr
from random import uniform
from shapely.geometry import shape, Point, Polygon
from PIL import Image

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def RandomPointFromBbox(bbox):
    l,b,r,t = bbox
    point = Point(uniform(l,r),uniform(t,b))
    return point.__geo_interface__['coordinates']

def GetPointsFromData(data, minPointCount = 100.0):
    points = []
    pointCount = max(float(minPointCount), len(data["features"]))
    geom = data["features"][0]["geometry"]
    polygon = shape(geom)
    bbox = polygon.bounds
    for i in range(minPointCount):
        points.append(RandomPointFromBbox(bbox))
    return points


In [ ]:
with open("list.json") as f:
    countryList = json.load(f)
    
with open("centroids.geojson") as f:
    centroids = json.load(f)

In [ ]:
for country in countryList:
    match = False
    for feature in centroids["features"]:
        for name in feature["properties"]["names"]:
            if country['name_en'] == name:
                match = True
                break
        if match:
            break
    if not match:
        print "Missing %s" % country["name_en"]

In [ ]:
unhcrData = []
with open("unhcr_popstats_export_time_series_all_data.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        unhcrData.append(row)

In [ ]:
refugees = {}
for row in unhcrData:
    year = row['Year']
    value = row['Value']
    org = row['Origin']
    dst = row['Country / territory of asylum/residence']
    if int(year) >= 1983:
        if row['Population type'] == 'Refugees (incl. refugee-like situations)':
            if org != 'Various/Unknown' and\
                dst != 'Various/Unknown' and\
                org != 'Stateless' and\
                dst != "Stateless":
                if org not in refugees:
                    refugees[org] = {dst: {'max': 0}}
                if dst not in refugees[org]:
                    refugees[org][dst] = {'max': 0}
                refugees[org][dst][year] = value if value != "*" else "0"


In [ ]:
countryNames = []
for org in refugees:
    countryNames.append(org)
    for dst in refugees[org]:
        countryNames.append(dst)
countryNames = list(set(countryNames))

In [ ]:
missing = []
countryName2Iso = {}
for country in countryNames:
    match = False
    for feature in centroids["features"]:
        for name in feature["properties"]["names"]:
            if unicode(country,"utf8").encode('ascii','ignore') == name:
                countryName2Iso[country] = feature["properties"]["iso_alpha-3"]
                match = True
                break
        if match:
            break
    if not match:
        print "Missing %s" % country
        missing.append(country)

In [ ]:
for org in refugees:
    for dst in refugees[org]:
        values = refugees[org][dst]
        for year in range(1984,2017):
            prevValue = int(values[str(year-1)]) if str(year-1) in values else 0
            currValue = int(values[str(year)]) if str(year) in values else 0
            if currValue - prevValue > values['max']:
                values['max'] = currValue - prevValue
            

In [ ]:
randomPoints = {}
for name in countryName2Iso:
    iso = countryName2Iso[name]
    filename = "gadm28_levels/%s/%s_adm0.geojson" % (iso,iso)
    if os.path.exists(filename):
        with open(filename) as f:
            geojson = json.load(f)
            points = GetPointsFromData(geojson, 1000)            
            randomPoints[iso] = points
    else:
        print "Missing %s" % filename
        l,b,r,t = (66, 23, 108, 42)
        points = []
        for i in range(1000):
            point = Point(uniform(l,r),uniform(t,b))
            points.append(point.__geo_interface__['coordinates'])
        randomPoints[iso] = points

In [ ]:
iso2Centroids = {}
for name in countryName2Iso:
    iso = countryName2Iso[name]
    match = False
    for feature in centroids["features"]:
        if feature["properties"]["iso_alpha-3"] == iso:
            match = True
            break
    iso2Centroids[iso] = feature["geometry"]["coordinates"]


In [ ]:
LonLatToPixelXY(iso2Centroids['AFG'])

In [ ]:
LonLatToPixelXY(randomPoints['AFG'][0])

In [ ]:
def xy2rgba(coords):
    x,y = coords
    _max = 1.0;
    _min = 0.0
    r = math.floor(255.0 * (x - _min) / (_max - _min));
    g = math.floor(255.0 * (y - _min) / (_max - _min));
    b = 0.0;
    a = 255.0;
    return [r,g,b,a]

def rgba2xy(rgba):
    return [rgba[0] / 255.0 + rgba[2], (rgba[1] + rgba[3]/255.0)/ 255.0]

In [ ]:
# Uniqueness test
t = 0
for keys in randomPoints:
    test = []
    for point in randomPoints[keys]:
        a = LonLatToPixelXY(point)
        test.append(xy2rgba((a[0]/255.,a[1]/255.)))
    
    unique_data = [list(x) for x in set(tuple(x) for x in test)]
    if len(unique_data) < 100:
        print keys, len(unique_data)
        t += 1
print t

In [ ]:
# Encode web mercator floats into rgba
def frac(x):
    return x - math.floor(x)

def float2rgba(value):
    x,y = math.floor(value), frac(value)
    x_max = 255.0
    x_min = 0.0
    y_min = 0.0
    y_max = 1.0
    r = math.floor(255.0 * (x - x_min) / (x_max - x_min));
    g = math.floor(255.0 * (y - y_min) / (y_max - y_min));
    b = 0.0;
    a = 255.0;
    return [r,g,b,a]

def rgba2float(rgba):
    value = [rgba[0] / 255.0 + rgba[2], (rgba[1] + rgba[3]/255.0)/ 255.0]
    return value[0] + value[1]


In [ ]:
t = 0
for keys in randomPoints:
    test2 = []
    for point in randomPoints[keys]:
        a = LonLatToPixelXY(point)
        test2.append(float2rgba(a[0]))    
    unique_data = [list(x) for x in set(tuple(x) for x in test2)]
    if len(unique_data) < 100:
        print keys, len(unique_data)
        t +=1
print t

In [ ]:
isoIdx = []
i = 0
for key in sorted(randomPoints.keys()):
    isoIdx.append({'iso': key, 'idx': i})
    i += 1 

In [ ]:
isoIdx

In [ ]:
centroidHash = {}
for feature in centroids['features']:
    if feature['properties']['iso_alpha-3'] not in centroidHash:
        key = feature['properties']['iso_alpha-3']
        centroidHash[key] = feature["geometry"]["coordinates"]

In [ ]:
centroids['features'][0]['properties']

In [ ]:
width = height = len(isoIdx)
img = Image.new('RGBA', (width,height))
png = img.load()
for idx in isoIdx:
    y = idx['idx']
    iso = idx['iso']
    a = LonLatToPixelXY(centroidHash[iso])
    rgba = xy2rgba((a[0]/255.,a[1]/255.))
    png[0,y] = (int(rgba[0]), int(rgba[1]), int(rgba[2]), int(rgba[3]))
    points = []
    for point in randomPoints[iso]:
        a = LonLatToPixelXY(point)
        points.append(xy2rgba((a[0]/255.,a[1]/255.)))
    unique_data = [list(x) for x in set(tuple(x) for x in points)]    
    for x in range(1,width):
        rgba = unique_data[x % len(unique_data)]
        png[x,y] = (int(rgba[0]), int(rgba[1]), int(rgba[2]), int(rgba[3]))
        
img.save("points.png")